In [1]:
import pandas as pd
import numpy as np

In [83]:
df = pd.read_csv("../../../data/enron-email-dataset/emails.csv")

In [95]:
df["date"] = df["message"].map(lambda x : x.split("\n")[1][5:])

In [ ]:
df["sender"] = df["message"].map(lambda x : x.split("\n")[2][5:].strip())
df["receiver"] = df["message"].map(lambda x : x.split("\n")[3][3:].strip())
df["subject"] = df["message"].map(lambda x : x.split("\n")[4][7:].strip())
df["content"] = df["message"].map(lambda x : " ".join(x.split("X-")[-1].split("\n")[1:]))
df["date"] = pd.to_datetime(df["date"])
df["time"] = df["date"].map(lambda x : x.time())
df["date"] = df["date"].map(lambda x : x.date())

In [ ]:
df.drop("message", axis=1, inplace=True)

In [ ]:
df.head()

In [11]:
df.to_csv("../../../data/enron-email-dataset/emails_clean.csv")

In [59]:
df = pd.read_csv("../../../data/enron-email-dataset/emails_clean.csv")

In [60]:
df.sort_values("date")

,Unnamed: 0,Unnamed: 0.1,file,date,sender,receiver,subject,content,time
239175,239175,239175,kean-s/calendar/untitled/1310.,1980-01-01,steven.kean@enron.com,kmagruder@newpower.com,: Re:,Restructuring Today quotes Jeff at length on r...,00:00:00
63935,63935,63935,dasovich-j/all_documents/4220.,1980-01-01,jeff.dasovich@enron.com,paul.kaufman@enron.com,: Re: Options the Governor's Considering,X-bcc: X-Folder: \Jeff_Dasovich_Dec2000\Notes...,00:00:00
207415,207415,207415,kaminski-v/all_documents/5077.,1980-01-01,vince.kaminski@enron.com,vince.kaminski@enron.com,: TO_DO,1. Ecopetrol,00:00:00
502301,502301,502301,whalley-l/discussion_threads/257.,1980-01-01,greg.whalley@enron.com,jeffrey.shankman@enron.com,: Re:,he's a trip,00:00:00
206985,206985,206985,kaminski-v/all_documents/4647.,1980-01-01,vince.kaminski@enron.com,duffie@stanford.edu,: Re: Full version,X-Origin: Kaminski-V X-FileName: vkamins.nsf ...,00:00:00
502383,502383,502383,whalley-l/discussion_threads/340.,1980-01-01,outlook.team@enron.com,ject: 3 - URGENT - TO PREVENT LOSS OF INFORMATION,rsion: 1.0,Critical Migration Information: 1. Your sched...,00:00:00
227775,227775,227775,kean-s/all_documents/1309.,1980-01-01,steven.kean@enron.com,kmagruder@newpower.com,: Re:,Restructuring Today quotes Jeff at length on r...,00:00:00
64637,64637,64637,dasovich-j/all_documents/596.,1980-01-01,jeff.dasovich@enron.com,susan.scott@enron.com,: Re: Pedersen inquiry,"Hey, you be talkin' to gas trash, counsel. ...",00:00:00
64655,64655,64655,dasovich-j/all_documents/611.,1980-01-01,jeff.dasovich@enron.com,"janel.guerrero@enron.com, kirk@cea.sfex.com",: Re: hey,"Yes on regulatory. Discussion, not power poin...",00:00:00
208235,208235,208235,kaminski-v/all_documents/629.,1980-01-01,vince.kaminski@enron.com,john.lavorato@enron.com,: VaR issues,X-Origin: Kaminski-V X-FileName: vkamins.nsf ...,00:00:00


In [61]:
from flask import Flask, request
from flask_restful import Resource, Api
from sqlalchemy import create_engine
from json import dumps
from flask.ext.jsonpify import jsonify
import csv, sqlite3
from dateutil.parser import parse
import re


In [62]:
con = sqlite3.connect("test.sqlite")
db_connect = create_engine('sqlite:///test.sqlite')
try:
    df.to_sql("emails_clean", db_connect,  if_exists='append', index=False)
except: 
    pass


OperationalError: (sqlite3.OperationalError) table emails_clean has no column named Unnamed: 0.1 [SQL: 'INSERT INTO emails_clean ("Unnamed: 0", "Unnamed: 0.1", file, date, sender, receiver, subject, content, time) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)'] [parameters: ((0, 0, 'allen-p/_sent_mail/1.', '2001-05-14', 'phillip.allen@enron.com', 'tim.belden@enron.com', ':', 'Here is our forecast', '23:39:00'), (1, 1, 'allen-p/_sent_mail/10.', '2001-05-04', 'phillip.allen@enron.com', 'john.lavorato@enron.com', ': Re:', "Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the b ... (487 characters truncated) ... e discussion format.    My suggestion for where to go is Austin.  Play golf and rent a ski boat and jet ski's.  Flying somewhere takes too much time.", '20:51:00'), (2, 2, 'allen-p/_sent_mail/100.', '2000-10-18', 'phillip.allen@enron.com', 'leah.arsdall@enron.com', ': Re: test', 'test successful.  w

Gestion des erreurs, 
- Tester si sender est vide
- Tests si des choses marchent pas bien 
- Tester si les trucs que je renvois sont vides


In [16]:
df.ix[275024, "receiver"]

C:\Users\mohamed.al-ani\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


'mike.grigsby@enron.com, keith.holst@enron.com, frank.ermis@enron.com,'

In [81]:
df[df['date'] > '2017-02-01']

,Unnamed: 0,Unnamed: 0.1,file,date,sender,receiver,subject,content,time
43811,43811,43811,campbell-l/inbox/1212.,2020-12-24,jody421@hotmail.com,undisclosed.recipients@mailman.enron.com,: Tired Of Searching For Love In All The Wrong...,"<html> <head><META HTTP-EQUIV=3D""Expires"" CONT...",23:36:06
43854,43854,43854,campbell-l/inbox/1258.,2020-12-29,jody421@hotmail.com,undisclosed.recipients@mailman.enron.com,: Jennifer Lopez - Nudity,<html> <head> <title>::::&gt; FREE LIFETIME VI...,20:53:46
508375,508375,508375,williams-j/deleted_items/578.,2024-05-26,pse6yl706@aloha.net,ject: (None),rsion: 1.0,<html> <head> <title>Untitled Document</title>...,10:49:57
517039,517039,517039,zufferli-j/inbox/quotes/33.,2044-01-04,cramer@cadvision.com,john.zufferli@enron.com,: trades,"Howdy, bom went out 35 at 35.5 Feb traded 32...",22:48:58
517040,517040,517040,zufferli-j/inbox/quotes/35.,2044-01-04,cramer@cadvision.com,john.zufferli@enron.com,: Trades,"BOM 5th to 31st traded 34, 33.5 , 33.5 and 3...",16:59:46
517042,517042,517042,zufferli-j/inbox/quotes/38.,2044-01-02,cramer@cadvision.com,john.zufferli@enron.com,: trades jan 2002,feb dec trades 37.5 feb dec LL went out 20 at...,23:46:00
517045,517045,517045,zufferli-j/inbox/quotes/41.,2043-12-28,cramer@cadvision.com,"linsider.jed@enron.com, john.zufferli@enron.co...","ichuk.craig@enron.com, baldwin.derrick@enron.com,","X-cc: X-bcc: X-Folder: \ExMerge - Zufferli, ...",19:34:12


In [82]:
app = Flask(__name__)
api = Api(app)
 

class sender(Resource):
    def get(self, sender_mail):
        if not re.match("[^@]+@[^@]+\.[^@]+", sender_mail):
            return("Not a valid email address. Please try again.")
        
        conn = db_connect.connect() # connect to database
        query = conn.execute("select date, time, sender, receiver, subject, content from emails_clean where sender = '%s' " %sender_mail  ) # This line performs query and returns json result
        result = {'mails found': [dict(zip(tuple (query.keys()) ,i)) for i in query.cursor]}
        return jsonify(result)

class recipient(Resource):
    def get(self, recipient_mails):
        conn = db_connect.connect() # connect to database
        result = {}
        mails = recipient_mails.split(",")
        for mail in mails:
            if not re.match("[^@]+@[^@]+\.[^@]+", mail):
                return("Not a valid email address. Please try again : xx@xx.xx, xx@xx.xx")
            
        for mail in mails:
            query = conn.execute("select date, time, sender, receiver, subject, content from emails_clean where receiver LIKE '%"+mail+"%'") # This line performs query and returns json result
            result[mail] = [dict(zip(tuple (query.keys()) ,i)) for i in query.cursor]
        return jsonify({"mails found" : result})

    
class date_range(Resource):
    def get(self, range_):
        conn = db_connect.connect() # connect to database
        dates = range_.split(",")
        
        #Checki if a good interval
        if len(dates) != 2:
            return("not a valid date interval, please enter a valid date interval : yyyy-mm-dd,yyyy-mm-dd")
        
        #Check if a valid date format
        for date in dates:
            try:
                parse(date)
            except: 
                return("not a valid date interval, please enter a valid date interval : yyyy-mm-dd,yyyy-mm-dd")
        
        query = conn.execute("select date, time, sender, receiver, subject, content from emails_clean where date BETWEEN '"+dates[0]+"' and '"+dates[1]+"'") # This line performs query and returns json result
        result  = {"mails found" : [dict(zip(tuple (query.keys()) ,i)) for i in query.cursor]}
        return jsonify(result)
    
api.add_resource(sender, '/sender/<sender_mail>') # Route_1
api.add_resource(recipient, '/recipient/<recipient_mails>') # Route_2
api.add_resource(date_range, '/date_range/<range_>') # Route_2



if __name__ == '__main__':
     app.run(port='5002')

 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Nov/2017 18:17:39] "GET /date_range/2017-02-01,2080-01-05 HTTP/1.1" 200 -


In [24]:
a = "j"
print("hello '% %d %'", 5)

hello '% %d %' 5


In [7]:
conn = sqlite3.connect('test.sqlite')
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print(name[0])

emails_clean
